In [3]:
from numpy.typing import ArrayLike
from scipy.integrate import solve_ivp
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
import numpy as np

# Constants we have

In [ ]:
# Rate of CD8+T-lysed tumor cell debris activation of CD8+ T-cells
rho0 = 1.245e-2

# Tumor size for half-maximal CD8+T-lysed debris CD8+ T activation
alpha_0 = 2.5036e-3

# Tumor size for half-maximal NK-lysed debirs NK activation
alpha_1 = 2.5036*10 5

# Rate of NK-lysed tumor cell debris activation of NK Cells
rho1 = 6.68 * 10**(-2) 

# Rate of CD8+ T-cell death due to tumor interaction
a4 = 3.422e-10 

# Rate of activated CD8+ T-cell turnover
k_cd = 9e-3

# Rate of chemotherapy-induced tumor death
a2 = 0.9

# Michaelis Menten Kinetics
i = 1

# Max tumor population
T_max = 2

# Natural turnover rate of natural killer cells (2 weeks)
k_NK = 1/14

# Natural turnover rate for epithelial cells lining ducts (40yr women)
k_N = 1/147

# Rate of NK cell death due to tumor interaction 𝑐𝑒𝑙𝑙𝑠−1/𝐷𝑎𝑦−1
a5 = 2.794*(10**(-13))

# Max epithelial cell count
N_max = T_max / (1/100)

In [17]:
10**8 / (0.03)

3333333333.3333335

In [ ]:


constants = [
    alpha_0, alpha_1,
    a0, a1, a2, a3, a4, a5,  # a0=a1
    b_CD,         # comes after we get E_c
    E_c, E_d,
    g_T, g_N,     # Growth rates
    i,
    k_N, k_CD, k_NK,
    N_max,
    rho_0, rho_1,
    r_NK, r_CD,
    T_max]         # TODO: 60% thingy

In [4]:
def state_equations(t:float, y:ArrayLike, u_interpolation:CubicSpline, constants:tuple|list):
    """Define the state evolution equations of our system.

    Parameters:
        - t (float) : the time
        - y (ArrayLike) : ndarray (4,) [T, N, CD, NK]
        - u_interpolation : a CubicSpline object containing the 
                            values of the control u_interpolation(t) = [D_c(t), D_d(t)]
    
    Returns:
    y_dot : ndarray (4,)
        the derivative of the T cell concentration and the virus concentration at time t
    """

    alpha_0, alpha_1, a0, a1, a2, a3, a4, a5, b_CD, E_c, E_d, g_T, g_N, i, k_N, k_CD, k_NK, N_max, rho0, rho1, r_NK, r_CD, T_max = constants
    D_c, D_d = u_interpolation(t)
    T, N, CD, NK = y

    T_prime = g_T*T*np.log(T_max/T) - a1*N*T - a2*NK*T - a3*CD*T - (E_c*D_c + (4/5)*E_d*D_d)*T 
    N_prime = g_N * N * np.log(N_max/N) - k_N*N - a0*N*T
    CD_prime = r_CD - k_CD*CD - rho0*CD*T**i/(alpha_0 + T**i) - a4*CD*T - b_CD*D_c*CD
    NK_prime = r_NK - k_NK*NK - (rho1*NK*T**i / (alpha_1 + T**i)) - a5*NK*T

    return np.array([T_prime, N_prime, CD_prime, NK_prime])

In [9]:
def costate_equations(t, y, u_interpolation, state_solution, constants):
    '''
    Parameters
    ---------------
    t : float
        the time
    y : ndarray (2,)
        the lambda values at time t
    u_interpolation : CubicSpline
        the values of the control u_interpolation(t) = [u1(t), u2(t)]
    state_solution : result of solve_ivp on state_equations with
    dense_output=True, i.e., state_solution.sol(t) = [T(t), V(t)]
    constants : a_1, a_2, b_1, b_2, s_1, s_2, mu, k, g, c, B_1, B_2, A_1, A_2

    Returns
    --------------
    y_dot : ndarray (4,)
        the derivative of lambda at time t
    '''

    alpha_0, alpha_1, a0, a1, a2, a3, a4, a5, b_CD, E_c, E_d, g_T, g_N, i, k_N, k_CD, k_NK, N_max, rho0, rho1, r_NK, r_CD, T_max = constants
    D_c, D_d = u_interpolation(t)
    T, N, CD, NK = state_solution.sol(t)
    p1, p2, p3, p4 = y

    p1_prime = -(-p1*g_T + p1*g_T*np.log(T_max/T) - p1*a1*N - p1*a2*NK-p1*a3*CD-p1*(E_c*D_c + (4/5)*E_d*D_d) 
                 - p2*a0*N + (-(alpha_0 + T**i)*(i*p3*rho0*CD*T**(i-1)) + (p3*rho0*CD*T**i)*(i*T**(i-1)))/(alpha_0 + T**i)**2 - p3*a4*CD
                 + (-(alpha_1 + T**i)*(i*p4*rho1*NK*T**(i-1)) + (p4*rho1*NK*T**i)*(i*T**(i-1)))/(alpha_1 + T**i)**2 -p4*a5*NK - 2*T)
    p2_prime = -(-p1*a1*T + p2*g_N*np.log(N_max/N) - p2*g_N - p2*k_N - p2*a0*T)
    p3_prime = -(-p1*a3*T - p3*k_CD - p3*(rho0*T**i)/(alpha_0 + T**i) - p3*a4*T - p3*b_CD*D_c)
    p4_prime = -(-p1*a2*T - p4*k_NK - p4*(rho1*T**i)/(alpha_1 + T**i) - p4*a5*T)

    return np.array([p1_prime, p2_prime, p3_prime, p4_prime])